In [9]:
import torch
import pandas as pd
import numpy as np
from sklearn.preprocessing import normalize

In [22]:
df = pd.read_csv('dataset.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3896 entries, 0 to 3895
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HotelID       3896 non-null   int64  
 1   Name Hotel    3896 non-null   object 
 2   Address       3896 non-null   object 
 3   Location      3896 non-null   object 
 4   Descriptions  3896 non-null   object 
 5   Rating        3896 non-null   float64
 6   IsEnglish     3896 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 186.6+ KB


In [4]:
df['Location'].unique()

array(['Phú Quốc', 'TP. Hồ Chí Minh', 'Đà Nẵng', 'Hà Nội', 'Nha Trang',
       'Hội An', 'Đà Lạt', 'Sa Pa', 'Huế', 'Vũng Tàu'], dtype=object)

In [5]:
import re

def get_special_chars(df):
    # Gộp tất cả text trong dataframe (chỉ các cột kiểu object hoặc string)
    text_data = ' '.join(df.select_dtypes(include=['object']).astype(str).values.flatten())
    
    # Tìm tất cả ký tự không phải chữ, số hoặc khoảng trắng
    special_chars = re.findall(r'[^\w\s]', text_data)
    
    # Lấy unique và sắp xếp
    unique_chars = sorted(set(special_chars))
    
    print(f"🔎 Có {len(unique_chars)} ký tự đặc biệt khác nhau trong dữ liệu:\n")
    print(' '.join(unique_chars))
    return unique_chars


In [6]:
specials = get_special_chars(df)

🔎 Có 37 ký tự đặc biệt khác nhau trong dữ liệu:

! " # % & ' ( ) * + , - . / : ; @ [ ] ` | ~ ­ ̀ ́ ̂ ̃ ̉ ̛ ̣ ​ – ’ “ ” ☆ ✩


In [10]:
import re
import unicodedata

In [8]:
def clean_vietnamese_text(text):
    if not isinstance(text, str):
        return ""
        
    text = unicodedata.normalize('NFC', text)
    
    text = text.lower()

    text = text.replace('%', ' phần trăm')
    
    text = re.sub(r'/\s*đêm', ' mỗi đêm', text, flags=re.IGNORECASE)
    text = re.sub(r"[^\w\s\-,.]", " ", text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [9]:
text = "xin chàO"
print(clean_vietnamese_text(text))

xin chào


# Embedding với Sentence-BERT

# Load Sentence-BERT

In [12]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# sbert_model = SentenceTransformer('keepitreal/vietnamese-sbert')

# Ghép các trường lại thành 1 text để embedding

In [11]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3896 entries, 0 to 3895
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HotelID       3896 non-null   int64  
 1   Name Hotel    3896 non-null   object 
 2   Address       3896 non-null   object 
 3   Location      3896 non-null   object 
 4   Descriptions  3896 non-null   object 
 5   Rating        3896 non-null   float64
 6   IsEnglish     3896 non-null   bool   
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 186.6+ KB
None


In [12]:
df.isnull().sum()

HotelID         0
Name Hotel      0
Address         0
Location        0
Descriptions    0
Rating          0
IsEnglish       0
dtype: int64

In [13]:
df["TextForEmbedding"] = df["Descriptions"].apply(clean_vietnamese_text)

# Sinh embedding cho toàn bộ dataset

In [14]:
embeddings = sbert_model.encode(df["TextForEmbedding"].tolist(), batch_size=32, show_progress_bar=True)
embeddings = normalize(embeddings, axis=1)  # chuẩn hóa theo hàng

Batches:   0%|          | 0/122 [00:00<?, ?it/s]

In [15]:
print("Embedding shape:", embeddings.shape)

Embedding shape: (3896, 768)


In [16]:
print("✅ Embedding done!")
print("Kiểu dữ liệu:", type(embeddings))     # numpy.ndarray
print("Shape:", embeddings.shape)            # (số_samples, số_chiều) ví dụ (1000, 384)
print("Dtype:", embeddings.dtype)            # float32 hoặc float64

✅ Embedding done!
Kiểu dữ liệu: <class 'numpy.ndarray'>
Shape: (3896, 768)
Dtype: float32


In [17]:
norms = np.linalg.norm(embeddings, axis=1)
print(norms[:10])

[1.0000001  1.         0.99999994 1.0000001  1.0000001  1.
 1.         0.99999994 1.         1.        ]


# Lưu vào Milvus

In [3]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection, utility, connections

# Kết nối đến Milvus server

In [14]:
connections.connect(alias="default",host="localhost",port="19530")
collection_name = "hotels_collection_mpnet_base_v2"

# Tạo collection schema

In [20]:
# Kiểm tra xem collection đã tồn tại chưa
if utility.has_collection(collection_name):
    print(f"⚠️ Collection '{collection_name}' đã tồn tại, xóa và tạo mới lại...")
    utility.drop_collection(collection_name)

# Tạo mới collection
print(f"🚀 Tạo mới collection '{collection_name}'...")

fields = [
    FieldSchema(name="HotelID", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="TextForEmbedding", dtype=DataType.FLOAT_VECTOR, dim=768),  # Sentence-BERT = 384, PhoBERT = 768
    FieldSchema(name="Location", dtype=DataType.VARCHAR, max_length=100), 
]

schema = CollectionSchema(fields=fields, description="Hotel dataset embeddings")

collection = Collection(name=collection_name, schema=schema)
print(f"✅ Collection '{collection_name}' đã được tạo mới thành công!")

🚀 Tạo mới collection 'hotels_collection_mpnet_base_v2'...
✅ Collection 'hotels_collection_mpnet_base_v2' đã được tạo mới thành công!


# Chuẩn bị dữ liệu để insert

In [21]:
hotel_ids = df["HotelID"].astype(int).tolist()
locations = df["Location"].astype(str).str.strip().tolist()

In [22]:
print(type(embeddings))
print(type(hotel_ids))
print(type(locations))

<class 'numpy.ndarray'>
<class 'list'>
<class 'list'>


# Insert

In [23]:
collection.insert([
        hotel_ids,
        embeddings.tolist(),
        locations,
    ])

collection.flush()
print(f"📊 Số lượng bản ghi: {collection.num_entities}")

📊 Số lượng bản ghi: 3896


# Tạo index

In [24]:
index_params = {
    "index_type": "HNSW",
    "metric_type": "COSINE",
    "params": {"M": 8, "efConstruction": 64}
}
# Tạo index cho field vector
collection.release()
collection.drop_index()
collection.create_index(field_name="TextForEmbedding", index_params=index_params)

Status(code=0, message=)